# 영상의 합성곱 계산
- 2-D 디지털 신호의 합성은 필터를 한 칸씩 옮기면서 영상과 겹쳐지는 부분을 모두 곱해 합치면 된다.

# Padding의 필요성
- 합성곱 연산 시, 필터(커널)의 크기에 따라 영상의 크기가 줄어드는 문제가 있다.

# Zero-Padding
- 크기가 (2N + 1)인 커널에 대해, 상하좌우에 N개의 Zero-Padding을 해주면 된다.
    > 투입과 출력의 크기를 똑같이 유지할 수 있음.

# Stride
- 합성곱 연산에서 커널을 이동시키는 거리를 Stride라고 하며, 크게 하면 출력의 크기가 줄어든다.

# 곱에서 합성곱으로
- 입력 뉴런 대신 입력 영상을, 가중치 대신 필터를, 곱 대신 합성곱을 사용하면 된다.
- 편향(Bias)은 그대로 동일하게 유지된다.

# 풀링 계층
- 여러 화소를 종합하여 하나의 화소로 변환하는 계층.
    > 풀링 계층을 통과하면 영상의 크기가 줄어들고, 정보가 종합된다.<br>

- 가장 많이 쓰이는 방법은 최댓값(Max-Pooling)과 평균(Average-Pooling)이다.
- 합성곱 신경망의 애플리케이션에 맞는 풀링 계층을 사용한다.

# CNN 학습 실습

### import modules

In [13]:
import tensorflow as tf
import numpy as np

In [19]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

### 하이퍼 파라미터

In [29]:
EPOCHS = 10

### 네트워크 구조

In [30]:
def MyModel():
    return Sequential([Conv2D(32, (3, 3), padding='same', activation='relu'), MaxPool2D(),
    Conv2D(64, (3, 3), padding='same', activation='relu'), MaxPool2D(),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')]) 
    # Filter 갯수, (3,3), padding = same : Conv-layer 통해도 zero 패딩통해 그대로 유지

### 데이터 불러오기

In [31]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# 차원을 하나 덧붙이기, NHWC
x_train = x_train[..., np.newaxis] 
x_test = x_test[..., np.newaxis]
# ...은 앞에 있는 모든 axis에 대해 전부다 포함한다는 의미임.

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

In [32]:
print(x_train.shape) # 3차원
print(x_train[0].shape) # 2차원
# 3차원에 차원을 하나 덧붙여주어야함.
# CNN을 학습할 때 데이터의 구조는 batch + height + width + channel
# 총 랭크가 4인 데이터셋을 필요로함.

(60000, 28, 28, 1)
(28, 28, 1)


### 모델 생성

In [33]:
model = MyModel()

model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

### 모델 학습

In [34]:
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.5407 - accuracy: 0.8026 - val_loss: 0.2836 - val_accuracy: 0.8960
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2537 - accuracy: 0.9080 - val_loss: 0.2894 - val_accuracy: 0.8948
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2076 - accuracy: 0.9240 - val_loss: 0.2570 - val_accuracy: 0.9083
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1727 - accuracy: 0.9348 - val_loss: 0.2497 - val_accuracy: 0.9134
Epoch 5/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1466 - accuracy: 0.9446 - val_loss: 0.2466 - val_accuracy: 0.9215
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1156 - accuracy: 0.9573 - val_loss: 0.2604 - val_accuracy: 0.9175
Epoch 7/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0945 - accuracy: 0.9639 - val_loss: 0.3014 - val_ac